# Import Libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pathlib
from PIL import Image
import cv2
import os
import glob

# Functions

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
def crop_face(image_path, output_size=(224, 224)):
    
    # Detects faces in an image and crops the image to only include the face.
    # The cropped face is resized to `output_size`.
    
    # Parameters:
    # - image_path: Path to the input image.
    # - output_size: Desired size of the output image.
    
    # Returns:
    # - Cropped and resized face image as a PIL Image object. Returns None if no face is detected.
    
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    
    if len(faces) == 0:
        return None  # No faces detected
    
    # For simplicity, only consider the first detected face
    for (x, y, w, h) in faces:
        face = img[y:y+h, x:x+w]
        face = cv2.resize(face, output_size)
        face_pil = Image.fromarray(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))
        return face_pil
    
    return None

# Data Loading + Pre-Processing

In [ ]:
data_dir = r"C:\Users\faiza\Documents\Uni\Year 3\Final Project\Code\images"
#The below is for the cropped images to be stored at
cropped_faces_dir = r"C:\Users\faiza\Documents\Uni\Year 3\Final Project\Code\faces"
if not os.path.exists(cropped_faces_dir):
    os.makedirs(cropped_faces_dir)

In [ ]:
if os.path.exists(cropped_faces_dir):
    pass
else:
    # Assume images are organized in subdirectories for each class
    for class_name in os.listdir(data_dir):
        class_dir = os.path.join(data_dir, class_name)
        if not os.path.isdir(class_dir):
            continue
        
        save_dir = os.path.join(cropped_faces_dir, class_name)
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        
        for img_file in glob.glob(class_dir + '/*.jpg'):  # Adjust the pattern as needed
            img_name = os.path.basename(img_file)
            save_path = os.path.join(save_dir, img_name)
            
            cropped_face = crop_face(img_file)
            if cropped_face is not None:
                cropped_face.save(save_path)
            else:
                img_file.save(save_path)

In [ ]:
data_dir = pathlib.Path(cropped_faces_dir).with_suffix('')

In [ ]:
data_dir

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

In [ ]:
angry = list(data_dir.glob('angry/*'))
PIL.Image.open(str(angry[0]))

In [ ]:
PIL.Image.open(str(angry[1]))

In [ ]:
happy = list(data_dir.glob('happy/*'))
PIL.Image.open(str(happy[0]))

In [ ]:
PIL.Image.open(str(happy[1]))

# Loading Data

In [ ]:
# batch size refers to the number of training examples utilised in one iteration. It will process 32 images at a time during training
batch_size = 32
#the height of the images will be restricted to 180
img_height = 180
#the width of the images will be restricted to 180
img_width = 180

In [ ]:
#begins the creation of the training dataset using tensorflow's keras api
#image_dataset_from_directory is used to load and preprocess the images so that it is suitable to train a model
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2, #80% of the data will be trained and 20% will be used to test it
    subset="training", #this specifies that it is for training
    seed=123, #random number generator to shuffle the dataset and create the split
    image_size=(img_height, img_width),
    batch_size=batch_size)

In [ ]:
#begins the creation of the validation dataset using tensorflow's keras api
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation", #this specifies that it is for validation
    seed=123, #this is the same to keep the split the same
    image_size=(img_height, img_width),
    batch_size=batch_size)

In [ ]:
#gets the class names
class_names = train_ds.class_names
print(class_names)

# Visualise Data

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

# Data Configuration

In [ ]:
#this optimises the performance of the training and validation datasets
#tf.data.AUTOTUNE prompts tensorflow to tune the buffering of elements in the data pipeline for optimal performance
#by setting it as a variable, it is being prepared to be used in the dataset operations that follow, allowing TensorFlow
#to dynamically adjust how much data to prefetch based on the current system's available resources
AUTOTUNE = tf.data.AUTOTUNE
#caches elements of train_ds. this loads images and labels into memory during the first epoch (iteration over dataset)
#this ensures them being reused in in subsequent epochs, speeding the training as I/O is reduced
#prefetch(buffer_size=AUTOTUNE) - prefetches the data; this is whilst the model is executing step 's' the input pipeline
#is reading step 's+1'
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
#creates a normalisation layer 1/255 is the factor each pixel will be multiplied. converts [0,255] to [0,1]
normalization_layer = layers.Rescaling(1./255)

In [ ]:
#maps the values [0,255] to [0,1]. that only applies to the x values and not the y values
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

# Creating Model (Keras)

In [ ]:
num_classes = len(class_names)
#Squential is a linear stack of layers which is one of the most simplest types of models in Keras. layers are just
#stacked in sequence
model = Sequential([
    layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)), #the first layer; reschales layer to normalise the pixel values of the input image
    layers.Conv2D(16, 3, padding='same', activation='relu'), #a 2D convolutional layer with 16 filters of size 3x3; RELU activation + padding='same'
    #means the output size will be the same as the input
    layers.MaxPooling2D(), #a max pooling layer reduces spatial dimensions of the input volume; helps with reducing the no. of parameters and computation
    layers.Conv2D(32, 3, padding='same', activation='relu'), #the depth of filters is increased
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(), #flattens the 3D output into a 1D array. this is because it will be used as the input to dense layers (which expect 1D arrays)
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
])

# Compile the Model

In [ ]:
#adam optimiser used. it is a popular choice for optimisation
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Model Summary

In [ ]:
model.summary()

# Train Model

In [ ]:
epochs=40
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

# Visualise Training Results

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Data Augmentation

In [ ]:
#Applies random transformations to the data. This improves the generalisation of the model and reduces overfitting
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal", #flips horizontally (left to right)
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.1), #randomly rotates the images by a random degree within the range [-10% * 2π, 10% * 2π]
    layers.RandomZoom(0.1), #it randomly zooms the images in and out by 10%
  ]
)

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")

# Dropout

In [ ]:
model = Sequential([
  data_augmentation, # will apply what happened in data augmentation from before
  layers.Rescaling(1./255), #normalises data after augmentation. scales the pixels to [0, 1]
  layers.Conv2D(16, 3, padding='same', activation='relu'), #convolutional and pooling layers are added for feature extraction
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2), #a dropout layer is added which converts 2D feature maps to a 1D feature vector, allowing it to be fed into the dense layers
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, name="outputs")
])

# Compile Model

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs = 60
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

# Visualise Training Results

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Predict on New Data

In [ ]:
# Load the face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load the image
pic_path = r"C:\Users\faiza\Downloads\1700725738962.jpeg"
img = cv2.imread(pic_path)

# Ensure the image was loaded
if img is None:
    raise ValueError("Failed to load image")

# Resize the image for faster face detection
img = cv2.resize(img, (640, int(img.shape[0] * (640/img.shape[1]))))
# Convert to grayscale for face detection
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Detect faces in the image
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

if len(faces) > 0:
    # Crop to the first detected face
    (x, y, w, h) = faces[0]
    face_img = img[y:y+h, x:x+w]

    # Display the cropped face image
    cv2.imshow('Cropped Face', face_img)
    cv2.waitKey(0)  # Wait for a key press to continue
    cv2.destroyAllWindows()

    # Optional: Save the cropped face image to a file
    cropped_face_path = r"C:\Users\faiza\Downloads\cropped_face.jpg"
    cv2.imwrite(cropped_face_path, face_img)

    # Prepare the cropped face image for model prediction
    face_img = cv2.resize(face_img, (img_height, img_width))
    face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB) / 255.0  # Normalize
    img_array = np.expand_dims(face_img, axis=0)
else:
    print("No face detected, using the whole image instead.")
    img = tf.keras.preprocessing.image.load_img(pic_path, target_size=(img_height, img_width))
    img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

test_img_array = np.random.rand(1, img_height, img_width, 3).astype(np.float32)  # Example random input
predictions = model.predict(test_img_array)  # Test prediction

# Predict with the model
predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])
predicted_class = class_names[np.argmax(score)]
confidence = 100 * np.max(score)

print(f"This image most likely belongs to {predicted_class} with a {confidence:.2f} percent confidence.")


In [ ]:
probabilities = tf.nn.softmax(predictions[0])
for i, prob in enumerate(probabilities):
    print(f"{class_names[i]}: {prob*100:.2f}%")

# Tensorflow Lite

## Tensorflow Keras Sequential Model to Tensorflow Lite Model

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
TF_MODEL_FILE_PATH = 'model.tflite' # The default path to the saved TensorFlow Lite model

interpreter = tf.lite.Interpreter(model_path=TF_MODEL_FILE_PATH)

In [ ]:
interpreter.allocate_tensors()

In [ ]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
input_shape = input_details[0]['shape']
input_dtype = input_details[0]['dtype']

In [ ]:
image_path = r"C:\Users\faiza\Downloads\WhatsApp Image 2024-02-15 at 10.53.10_5587b68b1.jpg"

In [ ]:
img = Image.open(image_path).resize((input_shape[1], input_shape[2]))
img_array = np.array(img)

In [ ]:
img_array = img_array / 255.0

In [ ]:
if img_array.ndim == 2:
    img_array = np.stack((img_array,) * 3, axis=-1)
elif img_array.shape[2] == 4:
    img_array = img_array[..., :3]

In [ ]:
img_array = np.expand_dims(img_array, axis=0).astype(input_dtype)

In [ ]:
interpreter.set_tensor(input_details[0]['index'], img_array)

In [ ]:
interpreter.invoke()

In [ ]:
output_data = interpreter.get_tensor(output_details[0]['index'])

In [ ]:
predicted_class = np.argmax(output_data, axis=1)
print(f"Predicted Class: {predicted_class}")

In [ ]:
# Assuming output_data is the output from your TFLite model
score_lite = tf.nn.softmax(output_data).numpy()
score_lite

In [ ]:
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score_lite)], 100 * np.max(score_lite))
)